In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
base_path = r'verify/base_verify1.csv'
money_path = r'verify/money_information_verify1.csv'
paient_path = r'verify/paient_information_verify1.csv'
report_path = r'verify/year_report_verify1.csv'
df_base = pd.read_csv(base_path,encoding='gbk')
df_money = pd.read_csv(money_path,encoding='gbk')
df_paient = pd.read_csv(paient_path,encoding='gbk')
df_report = pd.read_csv(report_path,encoding='gbk')
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30884 entries, 0 to 30883
Data columns (total 10 columns):
ID         30884 non-null int64
注册时间       30567 non-null float64
注册资本       30561 non-null float64
行业         30607 non-null object
区域         30599 non-null object
企业类型       30567 non-null object
控制人类型      30545 non-null object
控制人ID      30602 non-null float64
控制人持股比例    30569 non-null float64
flag       30578 non-null float64
dtypes: float64(5), int64(1), object(4)
memory usage: 2.4+ MB


In [3]:
print(df_base.shape,df_money.shape,df_paient.shape,df_report.shape)

(30884, 10) (92652, 10) (30884, 4) (92652, 11)


In [4]:
df_two = df_base.join(df_paient.set_index('ID'),on='ID')
df_three = df_two.join(df_money.set_index('ID'),on='ID')
df = pd.merge(df_three,df_report, on="ID")
df_two.shape

(30884, 13)

In [5]:
df = df[np.isnan(df['flag'])==False]
df.shape

(275193, 32)

## 三个表统计

In [6]:
df_new_report = df_report[df_report["year"]==2017]
df_new_report.shape
df_new_three = df_two.join(df_new_report.set_index('ID'),on='ID')
df_new_three.shape

(30884, 23)

In [7]:
df2=df_new_three.copy()
lens = len(df2)
lists = []
for i in range(lens):
    if True in pd.isna(df2.iloc[i]).values:
        lists.append(i)
df2=df2.drop(lists)
df2.head()

,ID,注册时间,注册资本,行业,区域,企业类型,控制人类型,控制人ID,控制人持股比例,flag,...,year,从业人数,资产总额,负债总额,营业总收入,主营业务收入,利润总额,净利润,纳税总额,所有者权益合计
0,1500001,2010.0,6680.0,商业服务业,山东,股份有限公司,自然人,1.0,0.89,1.0,...,2017.0,744.0,233800.0,454240.0,303940.0,243152.0,151970.0,-30394.0,0.0,-220440.0
1,1500321,2001.0,9330.0,商业服务业,广东,股份有限公司,企业法人,321.0,0.72,1.0,...,2017.0,942.0,298560.0,578460.0,1313664.0,788198.4,394099.2,-131366.4,0.0,-279900.0
3,1500614,2001.0,7730.0,工业,江西,集体所有制企业,企业法人,614.0,0.58,1.0,...,2017.0,265.0,123680.0,57975.0,210256.0,126153.6,21025.6,-21025.6,0.0,65705.0
4,1501057,2002.0,6840.0,工业,福建,农民专业合作社,自然人,1057.0,0.84,1.0,...,2017.0,270.0,102600.0,47880.0,30780.0,21546.0,15390.0,-3078.0,0.0,54720.0
5,1501449,2007.0,3570.0,零售业,湖北,合伙企业,自然人,1449.0,0.74,1.0,...,2017.0,594.0,107100.0,207060.0,492660.0,344862.0,49266.0,-49266.0,0.0,-99960.0


In [10]:

# print(train_length,len(X),test_length)
from sklearn.utils import shuffle  
df2 = shuffle(df2)  

df2.to_csv('train.csv', encoding='gbk',index=False)      


In [10]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
from sklearn.model_selection import train_test_split
features = ["企业类型", "区域", "控制人持股比例", "控制人类型","注册资本","行业",
            "专利","商标","著作权","从业人数","资产总额","负债总额","营业总收入",
                  "主营业务收入","利润总额","净利润","纳税总额","所有者权益合计"]
updata_features = ["注册资本","从业人数","资产总额","负债总额","营业总收入",
                  "主营业务收入","利润总额","净利润","纳税总额","所有者权益合计"]
min_max_scaler = preprocessing.MinMaxScaler()
df2[updata_features] = min_max_scaler.fit_transform(df2[updata_features])
Y = df2["flag"].values
X = pd.get_dummies(df2[features]).values
x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size=0.2)

In [11]:
from sklearn.linear_model import LogisticRegression  #逻辑回归
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
from sklearn.metrics import f1_score
f1_macro = f1_score(y_test,y_pred,average='macro')
print('f1_macro: {0}'.format(f1_macro))

c:\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


f1_macro: 0.6841121772582465


In [12]:
from sklearn.svm import SVC  #支持向量机                                #scikit-learn
from sklearn.tree import DecisionTreeClassifier  #决策树
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
f1_macro = f1_score(y_test,y_pred,average='macro')
print('f1_macro: {0}'.format(f1_macro))
model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
f1_macro = f1_score(y_test,y_pred,average='macro')
print('f1_macro: {0}'.format(f1_macro))

f1_macro: 0.8011845878583327


c:\python\python36\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


f1_macro: 0.5195753625858944


In [13]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
f1_macro = f1_score(y_test,y_pred,average='macro')
print('f1_macro: {0}'.format(f1_macro))

f1_macro: 0.8496461742805304


c:\python\python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [27]:
# from sklearn.decomposition import PCA
# pca = PCA(whiten=True,n_components=0.9,random_state=62).fit(x_train)
# x_train = pca.transform(x_train)
# x_test = pca.transform(x_test)
cls = MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=400, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
cls.fit(x_train, y_train)
y_pred = cls.predict(x_test)
f1_macro = f1_score(y_test,y_pred,average='macro')
print('f1_macro: {0}'.format(f1_macro))

f1_macro: 0.8120466110037778


(100,100)
max_iter=200   acc = 86.23
max_iter=300   acc = 86.95
max_iter=400   acc = 87.05
(100,100,50)
max_iter=200   acc = 86.04
max_iter=300   acc = 86.16
max_iter=400   acc = 86.91
(100,100,adam)
max_iter=200   acc = 82.05
max_iter=300   acc = 81.20
max_iter=400   acc = 81.22

In [17]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier  #决策树
# Adaboost分类器（多棵决策树）
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),n_estimators=40)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
f1_macro = f1_score(y_test,y_pred,average='macro')
print('f1_macro: {0}'.format(f1_macro))

f1_macro: 0.8683708126212453


In [41]:
from xgboost import XGBClassifier
model = XGBClassifier()               # 载入模型（模型命名为model)
model.fit(x_train,y_train)            # 训练模型（训练集）
y_pred = model.predict(x_test)   
f1_macro = f1_score(y_test,y_pred,average='macro')
print('f1_macro: {0}'.format(f1_macro))

f1_macro: 0.8797827931709133


In [40]:
from sklearn.naive_bayes import MultinomialNB
estimator = MultinomialNB()
estimator.fit(x_train,y_train)

y_pred = estimator.predict(x_test)   
f1_macro = f1_score(y_test,y_pred,average='macro')
print('f1_macro: {0}'.format(f1_macro))

f1_macro: 0.41187625933388644


c:\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
df2=df_new_three.copy()
lens = len(df2)
lists = []
for i in range(lens):
    if True in pd.isna(df2.iloc[i]).values:
        lists.append(i)
df2=df2.drop(lists)
df2['其它收入'] = df2['营业总收入']-df2['主营业务收入']

In [9]:

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
from sklearn.model_selection import train_test_split
features = ["企业类型", "区域", "控制人持股比例", "控制人类型","注册资本","行业",
            "专利","商标","著作权","从业人数","资产总额","负债总额","营业总收入",
                  "主营业务收入","利润总额","净利润","纳税总额","所有者权益合计",'其它收入']
updata_features = ["注册资本","从业人数","资产总额","负债总额","营业总收入",
                  "主营业务收入","利润总额","净利润","纳税总额","所有者权益合计",'其它收入']
min_max_scaler = preprocessing.MinMaxScaler()
df2[updata_features] = min_max_scaler.fit_transform(df2[updata_features])
Y = df2["flag"].values
X = pd.get_dummies(df2[features]).values
x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size=0.2)

In [10]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier  #决策树

# Adaboost分类器（多棵决策树）
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=40)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print("Adaboost分类器（多棵决策树）分类结果如下：")
accuracy = accuracy_score(y_test,y_pred)
print("accuarcy: %.2f%%" % (accuracy*100.0))

Adaboost分类器（多棵决策树）分类结果如下：
accuarcy: 88.39%


In [101]:
data = np.where(y_test!=y_pred)

In [11]:
train_length = int(len(X)*0.8)
test_length = len(X)-train_length
# x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size=0.2)
x_train,x_test = X[:train_length],X[train_length:]
y_train,y_test = Y[:train_length],Y[train_length:]

In [12]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier  #决策树

# Adaboost分类器（多棵决策树）
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=40)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print("Adaboost分类器（多棵决策树）分类结果如下：")
accuracy = accuracy_score(y_test,y_pred)
print("accuarcy: %.2f%%" % (accuracy*100.0))

Adaboost分类器（多棵决策树）分类结果如下：
accuarcy: 84.10%
